In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Fungsi untuk mengurai XML Pascal VOC
def parse_voc_xml(xml_path):
    import xml.etree.ElementTree as ET
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    objects = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        objects.append((name,))
    return objects

# Fungsi untuk memuat dataset
def load_dataset(dataset_dir):
    images = []
    labels = []
    label_map = {'document': 0, 'KTP': 1, 'KK': 2, 'SIM': 3}  # Pemetaan label ke angka
    for folder in ["train", "test", "valid"]:
        folder_path = os.path.join(dataset_dir, folder)
        for file in os.listdir(folder_path):
            if file.endswith(".xml"):
                xml_path = os.path.join(folder_path, file)
                objects = parse_voc_xml(xml_path)
                if len(objects) > 0:  # Pastikan file XML memiliki objek
                    image_name = file.replace(".xml", "") + ".jpg"  # Asumsi file gambar memiliki nama yang sama dengan file XML
                    image_path = os.path.join(folder_path, image_name)
                    if os.path.exists(image_path):
                        img = cv2.imread(image_path)
                        if img is not None:
                            img = cv2.resize(img, (256, 256))
                            images.append(img)
                            labels.append(label_map[objects[0][0]])  # Konversi label string ke angka
                        else:
                            print(f"[Warning] Gambar tidak dapat dibaca: {image_path}")
                    else:
                        print(f"[Warning] Path gambar tidak ditemukan: {image_path}")
    return np.array(images), np.array(labels)

# Membuat model CNN
def create_model():
    input_layer = Input(shape=(256, 256, 3))
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(4, activation='softmax')(x)  # Sesuaikan jumlah kelas
    model = Model(inputs=input_layer, outputs=output)
    return model

# Memuat dataset
dataset_dir = "./dataset"
images, labels = load_dataset(dataset_dir)

# Konversi labels menjadi one-hot encoding
labels = tf.keras.utils.to_categorical(labels, num_classes=4)  # 4 adalah jumlah kelas

# Memastikan bentuk labels adalah 2D (None, num_classes)
print("Shape of labels after to_categorical:", labels.shape)

# Membagi dataset menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Memastikan dimensi dataset sebelum melatih
print("Shapes before training:")
print("X_train shape:", X_train.shape)  # Harus (batch_size, 128, 128, 3)
print("y_train shape:", y_train.shape)  # Harus (batch_size, num_classes)
print("X_test shape:", X_test.shape)    # Harus (batch_size, 128, 128, 3)
print("y_test shape:", y_test.shape)    # Harus (batch_size, num_classes)

# Membuat model
model = create_model()

# Menyusun model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model.summary()

# Melatih model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Mengevaluasi model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


In [ ]:
# Menyimpan model ke file .h5
model.save('model.h5')
print("Model saved to model.h5")

In [ ]:
# Mengonversi model ke format .tflite
def save_tflite_model(model, output_path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(output_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model berhasil disimpan sebagai {output_path}")

# Simpan model ke dalam file .tflite
save_tflite_model(model, "model_klasifikasi.tflite")

In [ ]:
import matplotlib.pyplot as plt

# Memvisualisasikan hasil pelatihan
def plot_training_history(history):
    # Plot akurasi
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Panggil fungsi untuk menampilkan plot
plot_training_history(history)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np

# Fungsi untuk memuat model .tflite
def load_tflite_model(tflite_path):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    return interpreter

# Fungsi untuk melakukan preprocessing pada gambar
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Gambar tidak dapat dibaca dari path: {image_path}")
    img = cv2.resize(img, (256, 256))
    img = img.astype('float32') / 255.0  # Normalisasi
    img = np.expand_dims(img, axis=0)  # Tambahkan batch dimension
    return img

# Fungsi untuk menjalankan prediksi menggunakan model .tflite
def predict_image(tflite_model_path, image_path, label_map):
    # Memuat model .tflite
    interpreter = load_tflite_model(tflite_model_path)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Preprocessing gambar
    input_data = preprocess_image(image_path)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Jalankan model
    interpreter.invoke()

    # Ambil hasil prediksi
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_label = np.argmax(output_data)
    confidence = np.max(output_data)

    # Tampilkan hasil
    label_name = [k for k, v in label_map.items() if v == predicted_label][0]
    print(f"Prediksi: {label_name} (Confidence: {confidence:.2f})")


In [ ]:
# Label mapping (sama dengan yang digunakan saat melatih model)
label_map = {'document': 0, 'KTP': 1, 'KK': 2, 'SIM': 3}

# Path model .tflite dan gambar
tflite_model_path = "model_klasifikasi.tflite"
image_path = "99.jpg"  # Ganti dengan path gambar yang ingin diuji

# Jalankan prediksi
predict_image(tflite_model_path, image_path, label_map)